In [26]:
import binascii

class NovaCrypt:
    def __init__(self):
        self.block_size = 16 
        # S-Box: f(x) = (3x + 7) mod 256
        self.sbox = [(3 * i + 7) % 256 for i in range(256)]
        # Ters S-Box: f^-1(y) = 171 * (y - 7) mod 256
        self.inv_sbox = [(171 * (i - 7)) % 256 for i in range(256)]

    def anahtar_uret(self, parola):
        """Kullanıcı parolasını 16 byte anahtara dönüştürür"""
        p_bytes = parola.encode('utf-8')
        return (p_bytes * (16 // len(p_bytes) + 1))[:16]

    def _p_box(self, block, reverse=False):
        """Permütasyon: Byte kaydırma işlemi"""
        if not reverse:
            return block[1:] + block[:1]
        return block[-1:] + block[:-1]

    def sifrele(self, duz_metin, anahtar):
        """Düz metni şifreler"""
        if isinstance(duz_metin, str):
            duz_metin = duz_metin.encode('utf-8')
        
        # Padding: 16 byte'a tamamla
        data = duz_metin.ljust(self.block_size, b'\0')
        state = bytearray(data[:16])
        
        for i in range(4): # 4 Tur
            state = bytearray(a ^ b for a, b in zip(state, anahtar))
            state = bytearray(self.sbox[b] for b in state)
            if i < 3:
                state = self._p_box(state)
        return bytes(state)

    def desifrele(self, sifreli_metin, anahtar):
        """Şifreli metni deşifre eder"""
        state = bytearray(sifreli_metin)
        
        for i in reversed(range(4)): # Ters sırada dönmeli
            if i < 3:
                state = self._p_box(state, reverse=True)
            state = bytearray(self.inv_sbox[b] for b in state)
            state = bytearray(a ^ b for a, b in zip(state, anahtar))
            
        return bytes(state).rstrip(b'\0')

# --- TEST SENARYOLARI ---
nc = NovaCrypt()
parola = "ozgun_sifre_123"
anahtar = nc.anahtar_uret(parola)
mesaj = "Merhaba Dunya!"

print("--- Test 1: Basit Doğrulama ---")
sifreli = nc.sifrele(mesaj, anahtar)
cozulmus_bytes = nc.desifrele(sifreli, anahtar)

try:
    cozulmus_metin = cozulmus_bytes.decode('utf-8')
    print(f"Orijinal: {mesaj}")
    print(f"Şifreli (Hex): {sifreli.hex()}")
    print(f"Çözülmüş: {cozulmus_metin}")
except UnicodeDecodeError:
    print("Hata: Metin çözülemedi, anahtar veya algoritma hatalı!")

print("\n--- Test 2: Anahtar Hassasiyeti (Çığ Etkisi) ---")
hatali_anahtar = bytearray(anahtar)
hatali_anahtar[0] ^= 1 # Tek bir bit değiştirildi
cozulmus_hatali = nc.desifrele(sifreli, bytes(hatali_anahtar))

print(f"Orijinal Anahtar (Hex): {anahtar.hex()}")
print(f"Hatalı Anahtar (Hex):  {bytes(hatali_anahtar).hex()}")
print(f"Hatalı anahtar sonucu (Anlamsız Baytlar): {cozulmus_hatali.hex()}")
print("Sonuç: Tek bir bit değişikliği sonucu metin tamamen anlamsızlaştı.")




--- Test 1: Basit Doğrulama ---
Orijinal: Merhaba Dunya!
Şifreli (Hex): 976345f68f6f7f22b3fc60e307e0d4cb
Çözülmüş: Merhaba Dunya!

--- Test 2: Anahtar Hassasiyeti (Çığ Etkisi) ---
Orijinal Anahtar (Hex): 6f7a67756e5f73696672655f3132336f
Hatalı Anahtar (Hex):  6e7a67756e5f73696672655f3132336f
Hatalı anahtar sonucu (Anlamsız Baytlar): 4cb02bb56162612044756e796121
Sonuç: Tek bir bit değişikliği sonucu metin tamamen anlamsızlaştı.


In [25]:
import time
import hashlib

import struct # Sayıları bayt paketlerine çevirmek için daha profesyonel bir yol

class SecureNovaCrypt(NovaCrypt):
    def __init__(self):
        super().__init__()
        # S-Box'ı daha da güçlendirelim
        self.sbox = [((i * 31 + 17) ^ 0x63) % 256 for i in range(256)]

    def generate_otp(self, secret_key):
        # 1. Zamanı string yerine 8-byte'lık bir sayı paketi olarak al
        time_step = int(time.time() / 30)
        # 'q' formatı 8 byte (64-bit) integer demektir.
        state_bytes = struct.pack(">q", time_step) 
        
        # 16 byte'a tamamlamak için rastgele değil, sabit ama etkili padding
        state = state_bytes + b'\x55' * 8 
        
        key = self.anahtar_uret(secret_key)
        
        # 2. Tur sayısını 4'ten 8'e çıkaralım (Daha fazla karıştırma)
        temp_state = bytearray(state)
        for i in range(8): 
            temp_state = bytearray(a ^ b for a, b in zip(temp_state, key))
            temp_state = bytearray(self.sbox[b] for b in temp_state)
            temp_state = self._p_box(temp_state)
            
        cipher_block = bytes(temp_state)
        
        # 3. Dinamik Kırpma
        offset = cipher_block[-1] % 12 # 16 byte içinde güvenli bir aralık
        code_int = int.from_bytes(cipher_block[offset:offset+4], byteorder='big')
        
        return f"{(code_int % 1000000):06d}"

# --- KULLANIM ---
secure_nc = SecureNovaCrypt()
my_secret = "firat_yazilim_ozel_anahtar"

print(f"--- Güvenli OTP Üretici ---")
current_otp = secure_nc.generate_otp(my_secret)
print(f"Şu anki kodunuz: {current_otp}")
print(f"Kalan süre: {30 - (int(time.time()) % 30)} saniye")

--- Güvenli OTP Üretici ---
Şu anki kodunuz: 164044
Kalan süre: 29 saniye
